In [2]:
!pip install qiskit qiskit_aer

In [3]:
# Grover's Search Algorithm using Qiskit 2.x

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [4]:
def oracle(qc, n, marked_state):
    """Constructs the oracle for the marked state."""
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)  # use mcx() in Qiskit 2.x
    qc.h(n - 1)
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

In [5]:
def diffuser(qc, n):
    """Implements the Grover diffuser."""
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))

def grover_search(marked_state):
    """Builds Grover's search circuit for a given marked state."""
    n = len(marked_state)
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    oracle(qc, n, marked_state)
    diffuser(qc, n)
    qc.measure(range(n), range(n))
    return qc

In [6]:
def run_grover(qc):
    """Executes the Grover circuit and displays the result."""
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1024).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()

if __name__ == "__main__":
    marked_state = "101"
    print("Searching for marked state:", marked_state)
    qc = grover_search(marked_state)
    print(qc.draw(fold=-1))
    run_grover(qc)

Searching for marked state: 101
     ┌───┐          ┌───┐┌───┐               ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤┌───┐  │  ├───┤├───┤┌───┐       │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤├───┤┌─┴─┐├───┤├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫─┤M├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║ └╥┘
c: 3/════════════════════════════════════════════════════════╩══╩══╩═
                                                             0  1  2 


In [10]:
# === Grover's Search — All tasks, Qiskit 2.x compatible ===
# Run this entire cell in one go in Google Colab / Jupyter.

# Install required packages (only takes effect if missing)
# Remove the !pip lines if your environment already has these packages.
!pip install -q qiskit qiskit_aer pylatexenc

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from IPython.display import display

# --- Helper Functions ---

def oracle(qc: QuantumCircuit, n: int, marked_state: str):
    """Constructs the oracle for the marked state by flipping the phase of the marked basis state."""
    # Apply X to qubits where marked_state has '0' to map marked_state -> |11..1>
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)
    # Multi-controlled Z using mcx with an extra H on target to convert X<->Z
    qc.h(n - 1)
    if n - 1 > 0:
        qc.mcx(list(range(n - 1)), n - 1)  # controls = 0..n-2, target = n-1
    else:
        qc.x(0)  # trivial 1-qubit case (not used here)
    qc.h(n - 1)
    # Revert the X gates
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

def diffuser(qc: QuantumCircuit, n: int):
    """Implements the Grover diffuser (inversion about the mean)."""
    qubits = list(range(n))
    qc.h(qubits)
    qc.x(qubits)
    qc.h(n - 1)
    if n - 1 > 0:
        qc.mcx(list(range(n - 1)), n - 1)
    else:
        qc.x(0)
    qc.h(n - 1)
    qc.x(qubits)
    qc.h(qubits)

def grover_search(marked_state: str, iterations: int = 1) -> QuantumCircuit:
    """Builds Grover circuit for the given marked_state with specified iterations."""
    n = len(marked_state)
    qc = QuantumCircuit(n, n)
    # create uniform superposition
    qc.h(list(range(n)))
    # apply Grover iterations
    for _ in range(iterations):
        oracle(qc, n, marked_state)
        diffuser(qc, n)
    # measurement
    qc.measure(list(range(n)), list(range(n)))
    return qc

def run_grover(qc: QuantumCircuit, shots: int = 1024, noise_model=None, title: str = None):
    """Transpile, run the circuit on AerSimulator and plot histogram."""
    simulator = AerSimulator(noise_model=noise_model)
    compiled = transpile(qc, simulator)
    job = simulator.run(compiled, shots=shots)
    result = job.result()
    counts = result.get_counts()
    if title is None:
        title = "Grover Result"
    plot_histogram(counts, title=title)
    plt.show()
    return counts

# Utility to safely draw and display circuit (mpl if available else text)
def show_circuit(qc: QuantumCircuit):
    try:
        fig = qc.draw(output="mpl", fold=-1)
        display(fig)
    except Exception as e:
        # fallback to text if mpl drawer not available
        print(qc.draw(output="text"))

# === TASK 1: Try different marked states (3-qubit examples) ===
states = ["001", "010", "111"]
for state in states:
    print(f"\n🔍 Searching for marked state: {state}")
    qc = grover_search(state, iterations=1)
    show_circuit(qc)
    counts = run_grover(qc, title=f"Grover Search Result for {state}")

# === TASK 2: Extend to 4 qubits ===
marked_4 = "1010"
print(f"\nExtending to 4 qubits: marked state = {marked_4}")
qc4 = grover_search(marked_4, iterations=1)
show_circuit(qc4)
counts4 = run_grover(qc4, title=f"Grover Search Result for {marked_4}")

# === TASK 3: Multiple iterations (example: 3-qubit '101' with 2 iterations) ===
print("\nRunning multiple iterations for marked state '101' (2 iterations)")
qc_iter = grover_search("101", iterations=2)
show_circuit(qc_iter)
counts_iter = run_grover(qc_iter, title="Grover Search with 2 Iterations (101)")

# === TASK 4: Add Noise Model ===
print("\n⚡ Grover with a simple depolarizing noise model for '101'")
noise_model = NoiseModel()
# single-qubit depolarizing (applied to H and X gates for demo)
single_error = depolarizing_error(0.01, 1)
twoq_error = depolarizing_error(0.03, 2)
noise_model.add_all_qubit_quantum_error(single_error, ['h', 'x'])
noise_model.add_all_qubit_quantum_error(twoq_error, ['cx'])

qc_noise = grover_search("101", iterations=1)
show_circuit(qc_noise)
counts_noise = run_grover(qc_noise, noise_model=noise_model, title="Grover Search with Noise (101)")

print("\nAll tasks completed.")


🔍 Searching for marked state: 001
     ┌───┐┌───┐     ┌───┐┌───┐┌───┐          ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤├───┤  │  ├───┤├───┤├───┤       │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤├───┤┌─┴─┐├───┤├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫─┤M├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║ └╥┘
c: 3/════════════════════════════════════════════════════════╩══╩══╩═
                                                             0  1  2 

🔍 Searching for marked state: 010
     ┌───┐┌───┐          ┌───┐┌───┐┌───┐               ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤└───┘       │  ├───┤├───┤└───┘            │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├────────────■──┤ H ├┤ X ├─────────────────■──┤ X ├